# TP2 : Extraction d'information

## Imports

In [ ]:
# Keywords
import os
import yake

# Word Cloud
from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

# NER
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences
nlp = spacy.load('fr_core_news_md')

# Sentiment Analysis
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

## Création d'un fichier avec tous les journaux de l'année 1912

In [ ]:
year = 1912

# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]

# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding="utf-8") as f:
        content_list.append(f.read())

In [ ]:
# On peut compter le nombre d'éléments (= fichiers) dans la liste -> c'est (presque) toujours 100, ça permet donc de vérifier s'il n'y a pas eu de problème dans l'exécution du code
print(len(content_list))

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding="utf-8") as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding="utf-8") as f:
    file_year = f.read()

file_year[:500]

## Extraction de mots clés avec Yake

In [ ]:
# Instantier l'extracteur de mots clés en fr + les 50 premiers
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

# Récupérer le texte du temporaire de l'année étudiée, en mode lecture ('r'), en encodage utf-8
text = open(os.path.join(temp_path, f"{year}.txt"), 'r', encoding="utf-8").read()

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

In [ ]:
# Ne garder que les bigrammes / trigrammes et plus
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
    # if len(words) == 3:
    # if len(words) > 3 :
        kept.append(kw)
kept

## Nuage de mots

In [ ]:
# Stopwords
sw = stopwords.words("french")
sw += [
    # Coquilles OCR pas enlevées par fonction de nettoyage
    "dem", "serv", "chaus", "aujourd", "sach", "jard","brux","aven", "occas", "pens", "repr", "prés", "chamb", "culs",
    # Mots (quasiment) vides de sens
    "les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous", "celle", "entre", "encore", "toutes", "toute", "pendant", "moins", "dire", "voir", "cela", "non", "faut", "trois", "quatre", "cinq", "quart", "demi", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres", "van", "het", "autre", "jusqu", "très", "trop", "chez", "près", "toutes", "leurs", "avant", "suite", "rien", "quelques", "puis", "alors", "quand", "ceux", "elles", "déjà", "celui", "devant", "toujours", "outre", "tant", "mieux", "assez", "beaucoup", "plusieurs", "quelque", "quelques", "vers",
    # Mots pas pertinents dans le contexte du journal
    "rossel", "agence", "nord", "midi", "royale", "ville", "avenue", "place", "boulevard", "chaussée", "saint", "octobre", "mardi", "août", "dimanche", "septembre", "lundi", "décembre", "janvier", "juin", "avril", "mercredi", "samedi", "novembre", "jeudi", "vendredi"]
sw = set(sw)

# Fonction de nettoyage

def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 3 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
# Appliquer la fonction et vérifier les résultats

clean_text(year, folder=temp_path)

with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding="utf-8") as f:
    after = f.read()

# print(after[:500], "\n---")

# Calculer les mots les plus fréquents et afficher les résultats

frequencies = Counter(after.split())

print(frequencies.most_common(200))

In [ ]:
# Créer, stocker, afficher le nuage de mots

cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

## Reconnaissance d'entités nommées avec SpaCy

In [ ]:
# Charger le texte
n=100000
text_NER = open(os.path.join(temp_path, f"{year}.txt"), 'r', encoding="utf-8").read()[:n]

# Traiter le texte

doc = nlp(text_NER)

# Compter les entités (personnes, lieux, organisations)

people = defaultdict(int)
places = defaultdict(int)
orga = defaultdict(int)

for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    elif ent.label_ == "LOC" and len(ent.text) > 3:
        places[ent.text] += 1
    elif ent.label_ == "ORG" and len(ent.text) > 3:
        orga[ent.text] += 1

# Trier

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
sorted_places = sorted(places.items(), key=lambda kv: kv[1], reverse=True)
sorted_orga = sorted(orga.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
# Imprimer les 50 premières personnes

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

In [ ]:
# Imprimer les 50 premiers lieux

for place, freq in sorted_places[:50]:
    print(f"{place} apparait {freq} fois dans le corpus")

In [ ]:
# Imprimer les 50 premières organisations

for orga, freq in sorted_orga[:50]:
    print(f"{orga} apparait {freq} fois dans le corpus")

## Sentiment analysis avec Textblob-FR

In [ ]:
# Fonction

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

In [ ]:
# Exécuter la fonction sur des phrases choisies manuellement

get_sentiment("Trois d’entre elles purent être assez rapidement dégagées; quant au jeune Taelemans il fut tiré de sa situation critique sans connaissance.")
get_sentiment("Les champs sont sous eau, et tes fermiers repêchent ce qu’ils peuvent sauver du désastre.")
get_sentiment("La Fédération des jeunes-gardes libérales organise un congrès extraordinaire qui aura lieu le 24 novembre prochain, en vue de discuter la question de la révision.")
get_sentiment("Les coups de revolver et de couteau devaient faire une troisième victime.")
get_sentiment("Le Roi est en grande tenue de lieutenant- général, la Reine porte une merveilleuse toilette recouverte de véritable dentelle, ainsi que le manteau do dentelle qui lui fut offert par les •dames belges Jars de son mariage, et représentant les écuesons des neuf provlnces.")
get_sentiment("Voyant ses maigres ressources- épuisées, cl redoutant pardessus tout de retourner A lu colonie, le malheureux s’est suicidé hier soir en se pendant A la poignée de sa porte On n trouvé sur sa table un billet dans lequel U exposait, en quelques mots, les motifs do sa funeste détermination.")
get_sentiment("La presse allemande La limette de Yoss, ce matin, célèbre l'héroïsme du capitaine du Titanic, et considère que. 1e récit de sa mort est une belle page d’enseignement à lire aux enfants.")
get_sentiment("Si vous avez une gastrite, une dyspepsie, une gastralgie, une dilatation, une entérite, des indigestions,des vomissements; si après le repas vous avez des renvois, des lourdeurs, des aigreurs, des gonflements, des sur- focations.despalpitations, des maux de tête, de la somnolence; si vous ôtes devenu triste, mélancolique, neurasthénique, prenez les Poudres de Gock.")
get_sentiment("Mais, boudeuse, Lili lui refusait, sa main ' pour se cramponner au bras de Grenoult.")
get_sentiment("Par suite relations insuf. on dés. marier j. fille 22 a , jolie, bon. éducat., enfant unique, mère veuve, dot 150,000 fr.. plus bel. eapér- à M. dlstine- poslt. sfalsle, préfér. industr. Discrétion. Itettr. signées.")